In [1]:
import pickle
import numpy as np

features_min, features_max = np.load("norm_constants.npy",allow_pickle=True).tolist()

with open('cluster_year.pkl', 'rb') as f:
    cluster_year = pickle.load(f)

with open('name_encoder.pkl', 'rb') as f:
    le = pickle.load(f)


## LSTM/GRU

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


def printf(*args, fname="log.txt"):
    with open(os.path.join("test_outputs",fname),"a+") as f:
        for a in args:
            f.write(str(a) + " ")
        f.write("\n") 
    print(args) 

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import tensorflow as tf

import optuna
import os

2025-08-23 01:34:21.114840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755902061.527804  839447 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755902061.696674  839447 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755902064.428662  839447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755902064.428699  839447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755902064.428700  839447 computation_placer.cc:177] computation placer alr

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import regularizers
from tkan import TKAN
from tensorflow.keras.models import load_model
from tensorflow.keras import ops
from tensorflow.keras import Loss
from tensorflow import keras
#np.bool = np.bool_

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)
targs = []
datas = []

for c, y in cluster_year:
    for j,k in cluster_year[c,y][1]:
        targs += [cluster_year[c,y][1][j,k][1]]
        datas += [cluster_year[c,y][1][j,k][0]]

X = np.vstack(datas).reshape(-1,5,4)
y = np.vstack(targs)

X_train, X_test, ynn_train,ynn_test = train_test_split(X,y,test_size = 0.33)

def make_modelLSTM(input_shape, hidden_size, dropout):
    input_layer = tf.keras.layers.Input(input_shape)
    #dim = tf.zeros([batch_size,hidden_size])  
    output_layer = tf.keras.layers.LSTM(hidden_size, kernel_regularizer=regularizers.L2(0.001),return_sequences=True,dropout=dropout)(input_layer)#, initial_state=[dim, dim])
    output_layer2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='tanh'))(output_layer)    
    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer2)
 
def make_GRU(input_shape, hidden_size, dropout):
    input_layer = tf.keras.layers.Input(input_shape)
    output_layer = tf.keras.layers.GRU(hidden_size, kernel_regularizer=regularizers.L2(0.001),return_sequences=True,dropout=dropout)(input_layer)#, initial_state=[dim, dim])
    output_layer2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='tanh'))(output_layer)    
    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer2)

def make_modelTKAN(input_shape, hidden_size, dropout):
    model = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=input_shape),
          TKAN(hidden_size, sub_kan_configs=[{'spline_order':3}], return_sequences=True, use_bias=True, kernel_regularizer=regularizers.L2(dropout),
    bias_regularizer=regularizers.L2(dropout), recurrent_regularizer=regularizers.L2(dropout)),
          tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='tanh')),
    ])
    return model

models = {"LSTM":make_modelLSTM, "GRU": make_GRU, "TKAN": make_modelTKAN}
#models = {"TKAN": make_modelTKAN}

for model_name in models:
    make_model = models[model_name]
    batch_size = 64       

    def objective(trial):
        lr = trial.suggest_float('lr', 0.00001, 0.01)
        hidden_size = trial.suggest_int('hs', 2, 32)

        if model_name == "TKAN":
            do = trial.suggest_float('dropout', 1e-5, 1e-2)
            ep = 50
        else:    
            do = trial.suggest_float('dropout', 0.05, 0.2)
            ep = 1000
            
        kf = KFold(n_splits=3)
        scores = []
        for _, (train_index, test_index) in enumerate(kf.split(X_train)):
            model = make_model(input_shape=X_train.shape[1:],hidden_size=hidden_size, dropout = do)
        # 
            opt = tf.keras.optimizers.Adam(learning_rate=lr)
            # callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
            #                                patience=10)

            checkpoint_filepath = 'checkpoint.keras'
            model_checkpoint_callback = ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_best_only=True,
                monitor='loss',  # Or 'val_accuracy', or any other metric
                mode='min',  # Or 'max' if you are monitoring a metric to be maximized
            )
            
            reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                                          patience=10, min_lr=0.00001)        
            
            model.compile(
                 optimizer=opt,
                 loss='binary_crossentropy',
                 metrics=["mean_squared_error"],
            )

            history = model.fit(
                 X_train[train_index],
                 ynn_train[train_index],
                 batch_size=batch_size,
                 epochs=ep,
                 verbose=0,callbacks=[model_checkpoint_callback, reduce_lr]
            )

            best_model = load_model(checkpoint_filepath)
            try:
                y_pred = best_model.predict(X_train[test_index]) #, batch_size=batch_size)
                scores.append(mean_squared_error(ynn_train[test_index].flatten(),y_pred.flatten()))
            except Exception as e:
                print(e)
                scores.append(1e26)
            del history
            del model    
        return np.asarray(scores).mean() 
        
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)    
    
    lr = study.best_trial.params["lr"]     
    hs = study.best_trial.params["hs"]     
    do = study.best_trial.params["dropout"]   

    model = make_model(input_shape=X_train.shape[1:],hidden_size=hs, dropout = do)
# 
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    #callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
    #                               patience=10)

    model.compile(
         optimizer=opt,
         loss='binary_crossentropy',
         metrics=["mean_squared_error"],
    )

    if model_name == "TKAN":
        ep = 50
    else:
        ep = 1000    

    checkpoint_filepath = 'checkpoint.keras'
    model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='loss',  # Or 'val_accuracy', or any other metric
        mode='min',  # Or 'max' if you are monitoring a metric to be maximized
    )

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                                  patience=10, min_lr=0.00001)  
    
    history = model.fit(
         X_train,
         ynn_train,
         batch_size=batch_size,
         epochs=ep,
         verbose=0,callbacks=[model_checkpoint_callback, reduce_lr]
    )   
    model = load_model(checkpoint_filepath)   
    try:
        y_pred = model.predict(X_test) #, batch_size=batch_size)
        mse_score = mean_squared_error(ynn_test.flatten() * features_max[0],y_pred.flatten() * features_max[0])
        mae_score = mean_absolute_error(ynn_test.flatten() * features_max[0],y_pred.flatten() * features_max[0])
        r2_ = r2_score(ynn_test.flatten(),y_pred.flatten())
        printf(model_name,mse_score, mae_score, r2_,fname="baseline_output.txt")     
        #nn_data.append([model_name,mse_score, mae_score])
    except Exception as e:
        print(e)
    del model

[I 2025-08-23 01:34:37,990] A new study created in memory with name: no-name-23fb512d-d7dc-4037-9d7c-cae7aab592c4
2025-08-23 01:34:38.023664: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


[I 2025-08-23 01:36:26,400] Trial 0 finished with value: 0.004038883028835872 and parameters: {'lr': 0.0029634807873267117, 'hs': 15, 'dropout': 0.07762635886697528}. Best is trial 0 with value: 0.004038883028835872.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


[I 2025-08-23 01:38:08,368] Trial 1 finished with value: 0.00396357945969137 and parameters: {'lr': 0.003854664623415963, 'hs': 10, 'dropout': 0.15720289740269122}. Best is trial 1 with value: 0.00396357945969137.


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step


[I 2025-08-23 04:48:44,831] Trial 48 finished with value: 0.2130224327130491 and parameters: {'lr': 0.004087334607276333, 'hs': 23, 'dropout': 0.001686669779366094}. Best is trial 7 with value: 0.003193324747322989.


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step


[I 2025-08-23 04:49:06,589] Trial 49 finished with value: 0.01737380534398426 and parameters: {'lr': 0.008252115486885453, 'hs': 11, 'dropout': 0.008966401355487779}. Best is trial 7 with value: 0.003193324747322989.


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
('TKAN', 2.7461739803763886e+18, 1417101712.8852923, -2.9323622283000943)


# Transformer regressor

In [ ]:
import time
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import warnings
import copy

warnings.filterwarnings("ignore")

for fft in [True, False]:
    
    max_cluster_size = 0
    
    lap_pos_enc = True
    wl_pos_enc = True
    device = "cpu"
    batch_size = 8
    epochs = 1000
    
    root_log_dir = ""
    root_ckpt_dir = "checkpoints"
    write_file_name = ""
    write_config_file = ""
    min_lr = 0.00000001
    if fft:
        cluster_year_ = copy.deepcopy(cluster_year)
        for c,y in cluster_year_:
            for j,k in cluster_year_[(c,y)][1]: 
                f1 = torch.fft.rfft2(torch.from_numpy(cluster_year_[(c,y)][1][j,k][0][:,:2]),norm="ortho")
                f2 = torch.fft.rfft2(torch.from_numpy(cluster_year_[(c,y)][1][j,k][0][:,2:4]),norm="ortho")
                #print(f1.shape,f2.shape,cluster_year_[(c,y)][1][j,k][0].shape)
                cluster_year_[(c,y)][1][j,k][0] =  torch.hstack([f1,f2])
    else:
        cluster_year_ = cluster_year
    
    dataset = TradeDGL(cluster_year_,device)
    net_params = {}
    
    net_params['fft'] = fft
    net_params['num_trade_indicators'] = 1
    net_params['hidden_dim'] = 4
    net_params['n_heads'] = 2
    net_params['out_dim'] = 1
    net_params['in_feat_dropout'] = 0.0
    net_params['dropout'] = 0.05
    net_params['L'] = 2
    net_params['layer_norm'] = False
    net_params['batch_norm'] = False
    net_params['residual'] = True
    net_params['edge_feat'] = True
    net_params['device'] = device
    net_params['lap_pos_enc'] = lap_pos_enc
    net_params['wl_pos_enc'] = wl_pos_enc
    net_params['max_cluster_size'] = len(dataset.all_countries)
    net_params['pos_enc_dim'] = 4
    
    
    net_params['num_states'] = len(dataset.all_countries)
    
    train_len = int(dataset.n_samples * 0.6)
    val_len = int(dataset.n_samples * 0.2)
    
    
    test_len = dataset.n_samples - train_len - val_len
    
    if lap_pos_enc:
        st = time.time()
    
        
        print("[!] Adding Laplacian positional encoding.")
        dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
        print('Time LapPE:',time.time()-st)
        
    if wl_pos_enc:
        st = time.time()
        print("[!] Adding WL positional encoding.")
        dataset._add_wl_positional_encodings()
        print('Time WL PE:',time.time()-st)
    
    trainset, valset, testset = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])
    
    
    model = GraphTransformerNet(net_params)
    
    model = model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                     factor=0.5,
                                                     patience=30,
                                                     verbose=True)
    
    epoch_train_losses, epoch_val_losses = [], []
        
    
    train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, collate_fn=dataset.collate)
    val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, collate_fn=dataset.collate)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, collate_fn=dataset.collate)
    min_loss = 100
    min_val = 0
    min_name = ""    
    try:
        with tqdm(range(epochs)) as t:
            for epoch in t:
    
                t.set_description('Epoch %d' % epoch)
    
                start = time.time()
    
                epoch_train_loss, optimizer = train_epoch(model, optimizer, device, train_loader, epoch)
                    
                epoch_val_loss = evaluate_network(model, device, val_loader, epoch)
    
                # Saving checkpoint
                ckpt_dir = os.path.join(root_ckpt_dir, "RUN_")
                if not os.path.exists(ckpt_dir):
                    os.makedirs(ckpt_dir)
                    
                if epoch_train_loss < min_loss:
                    min_loss = epoch_train_loss
                    min_val = epoch_val_loss
                    min_name = '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch))
                    print(min_loss)
                    
                epoch_train_losses.append(epoch_train_loss)
                epoch_val_losses.append(epoch_val_loss)
                t.set_postfix(time=time.time()-start, lr=optimizer.param_groups[0]['lr'],
                              train_loss=epoch_train_loss, val_loss=epoch_val_loss)
                
                #per_epoch_time.append(time.time()-start)
    
    
                torch.save(model.state_dict(), '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch)))
    
    
                scheduler.step(epoch_val_loss)
    
                #if optimizer.param_groups[0]['lr'] < min_lr:
                #    print("\n!! LR EQUAL TO MIN LR SET.")
                #    break
                
    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early because of KeyboardInterrupt')
    
    print("Convergence Time (Epochs): {:.4f}".format(epoch))
    
    print("Min loss:", min_loss,min_val)
    
    model = GraphTransformerNet(net_params)
    model.load_state_dict(torch.load(min_name, weights_only=True))
    sq,a,r,residuals = test_network(model, device, test_loader)
    printf('Transformer: encoder',fft,sq,a,r)
#writer.close()

[!] Adding Laplacian positional encoding.
Time LapPE: 9.05491590499878
[!] Adding WL positional encoding.
Time WL PE: 1.41568922996521


Epoch 1:   0%| | 1/1000 [00:00<09:20,  1.78it/s, lr=0.01, time=0.557, train_loss

0.6405973219411465


Epoch 2:   0%| | 2/1000 [00:01<08:41,  1.91it/s, lr=0.01, time=0.492, train_loss

0.33981088984889085


...

Epoch 43:   4%| | 43/1000 [00:29<15:50,  1.01it/s, lr=0.01, time=0.687, train_lo

0.1412120588713818


Epoch 51:   5%| | 51/1000 [00:33<10:20,  1.53it/s, lr=0.01, time=0.757, train_lo

0.13873865460713197


Epoch 65:   6%| | 65/1000 [00:42<10:28,  1.49it/s, lr=0.01, time=0.566, train_lo

# Decoder

In [9]:
import time
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

max_cluster_size = 0

lap_pos_enc = True
wl_pos_enc = True
device = "cpu"
batch_size = 8
epochs = 3000

root_log_dir = ""
root_ckpt_dir = "checkpoints"
write_file_name = ""
write_config_file = ""
min_lr = 0.00000001

cy_train, cy_test = train_test_split(list(cluster_year.keys()),train_size=0.8)

train_dataset = TradeDGLDecoder({k:cluster_year[k] for k in cy_train},device,random_ratio = 0.5)
test_dataset = TradeDGLDecoder({k:cluster_year[k] for k in cy_test},device,random_ratio = 1.0)
net_params = {}

net_params['fft'] = False
net_params['num_trade_indicators'] = 1
net_params['hidden_dim'] = 4
net_params['n_heads'] = 2
net_params['out_dim'] = 1
net_params['in_feat_dropout'] = 0.
net_params['dropout'] = 0.0
net_params['encL'] = 1
net_params['decL'] = 1
net_params['layer_norm'] = False
net_params['batch_norm'] = False
net_params['residual'] = True
net_params['edge_feat'] = True
net_params['device'] = device
net_params['lap_pos_enc'] = lap_pos_enc
net_params['wl_pos_enc'] = wl_pos_enc
net_params['max_cluster_size'] = max(len(train_dataset.all_countries),len(test_dataset.all_countries))
net_params['pos_enc_dim'] = 4


net_params['num_states'] = max(len(train_dataset.all_countries),len(test_dataset.all_countries))

if lap_pos_enc:
    st = time.time()

    
    print("[!] Adding Laplacian positional encoding.")
    train_dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
    test_dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
    print('Time LapPE:',time.time()-st)
    
if wl_pos_enc:
    st = time.time()
    print("[!] Adding WL positional encoding.")
    train_dataset._add_wl_positional_encodings()
    test_dataset._add_wl_positional_encodings()
    print('Time WL PE:',time.time()-st)

trainset, valset = torch.utils.data.random_split(train_dataset, [0.8,0.2])


model = GraphTransformerNetDec(net_params)
#model = SimpleGRU(net_params['hidden_dim'])
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                 factor=0.5,
                                                 patience=50,
                                                 verbose=True)

epoch_train_losses, epoch_val_losses = [], []
    

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, collate_fn=train_dataset.collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate)
min_loss = 100000
min_val = 0
min_name = ""    
try:
    with tqdm(range(epochs)) as t:
        for epoch in t:

            t.set_description('Epoch %d' % epoch)

            start = time.time()

            epoch_train_loss, optimizer = train_epoch(model, optimizer, device, train_loader, epoch)
                
            epoch_val_loss = evaluate_network(model, device, val_loader, epoch)

            # Saving checkpoint
            ckpt_dir = os.path.join(root_ckpt_dir, "RUN_")
            if not os.path.exists(ckpt_dir):
                os.makedirs(ckpt_dir)
                
            if epoch_train_loss < min_loss:
                min_loss = epoch_train_loss
                print(min_loss,epoch_val_loss)
                min_val = epoch_val_loss
                min_name = '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch))
                
            epoch_train_losses.append(epoch_train_loss)
            epoch_val_losses.append(epoch_val_loss)
            t.set_postfix(time=time.time()-start, lr=optimizer.param_groups[0]['lr'],
                          train_loss=epoch_train_loss, val_loss=epoch_val_loss)
            

            torch.save(model.state_dict(), '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch)))


            scheduler.step(epoch_val_loss)

except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early because of KeyboardInterrupt')

print("Convergence Time (Epochs): {:.4f}".format(epoch))

print("Min loss:", min_loss,min_val)

model = GraphTransformerNetDec(net_params)
model.load_state_dict(torch.load(min_name, weights_only=True))

#writer.close()

[!] Adding Laplacian positional encoding.
Time LapPE: 5.278119087219238
[!] Adding WL positional encoding.
Time WL PE: 1.0828883647918701


Epoch 1:   0%| | 1/3000 [00:08<7:05:44,  8.52s/it, lr=0.01, time=8.51, train_los

0.5815591007623894 0.5326838215561298


Epoch 2:   0%| | 2/3000 [00:16<7:02:29,  8.46s/it, lr=0.01, time=8.41, train_los

0.45648708532822724 0.39255612040856935


...

Epoch 1039:  35%|▎| 1039/3000 [2:21:31<4:24:57,  8.11s/it, lr=3.81e-8, time=8.07

0.12755724235993265 0.24149728175280044


Epoch 2999: 100%|█| 3000/3000 [6:47:09<00:00,  8.14s/it, lr=1.91e-8, time=8.3, t

Convergence Time (Epochs): 2999.0000
Min loss: 0.12755724235993265 0.24149728175280044


<All keys matched successfully>

In [10]:
sq,a,r,residuals,_ = test_network(model, device, test_loader)
printf('Transformer: encoder-decoder',sq,a,r)

[6.67143422e-03 2.32827331e-02 2.76407730e-01 1.63385886e-02
 2.19863103e-01 1.13026799e-01 7.80935515e-03 3.59803578e-03
 4.31692693e-02 1.46970502e-02 1.23295891e-02 6.61448770e-04
 4.65270111e-02 8.93450713e-02 1.60435914e-02 3.76402024e-04
 2.85426598e-04 3.53031983e-04 1.58078292e-02 2.32046415e-02
 4.86398262e-02 7.76321327e-02 2.91139900e-03 5.03105253e-02
 1.84605005e-02 3.54295211e-01 1.25459034e-01 4.67457258e-02
 1.17590546e-01 1.64828041e-02] [0.01748795 0.06323804 0.00997239 0.00718756 0.16782949 0.05457494
 0.00213992 0.00284901 0.03818799 0.00343212 0.00511989 0.00198032
 0.05160219 0.03539711 0.0454592  0.00158577 0.00161337 0.00163251
 0.00435363 0.03110227 0.00722455 0.06322904 0.0020618  0.01129585
 0.02538064 0.2565765  0.00175113 0.04249722 0.10644878 0.00417612]
('Transformer: encoder-decoder', 9.714085832707962e+16, 142403652.58810768, 0.8695477575437767)
